# PART : 1

## Import all the Libraries

In [1]:
!pip install pandas
import pandas as pd
import numpy as np
!pip install BeautifulSoup4
!pip install lxml
!pip install html5lib
from pandas.io.html import read_html
print('Everything Imported')

     |████████████████████████████████| 102kB 2.8MB/s ta 0:00:011
     |████████████████████████████████| 5.8MB 2.7MB/s eta 0:00:01
Everything Imported


### Use Pandas to import table from Wikipedia page

In [112]:
wikipage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(wikipage, attrs={'class':'wikitable'})
wikitable[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [113]:
table = wikitable[0]
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop all the Rows where Borough Name is not assigned

In [114]:
table.drop(table[table.Borough == 'Not assigned'].index, inplace = True)
table.reset_index(drop = True, inplace = True)
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Rename all the unnamed Neighbourhoods to their respective Borough names

In [115]:
for a in table['Neighbourhood'].index:
    if table.iloc[a,2] == 'Not assigned':
        table.iloc[a,2] = table.iloc[a,1]

In [116]:
table.dropna(how = 'all', axis = 0, inplace = True)
table.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Merge the Neighbourhoods based on their Postcodes

In [117]:
toronto = table.groupby(['Postcode','Borough']).Neighbourhood.agg([', '.join])

In [118]:
toronto.reset_index(inplace = True)
toronto.rename(columns = {'join' : 'Neighbourhood'}, inplace = True)

In [119]:
toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Shape of the DataFrame

In [120]:
toronto.shape

(103, 3)

# PART: 2

### We read the Geospatial Coordinates file of Toronto Postal Codes in a Pandas DataFrame

In [122]:
Geospatial = pd.read_csv('Geospatial_Coordinates.csv')

In [123]:
Geospatial = Geospatial.sort_values('Postal Code', ascending = True)
Geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [124]:
toronto['Latitude'] = ''
toronto['Longitude'] = ''
for a in toronto['Postcode'].index:
    for b in Geospatial['Postal Code'].index:
        if toronto.iloc[a,0] == Geospatial.iloc[b,0]:
            toronto.iloc[a,3] = Geospatial.iloc[b,1]
            toronto.iloc[a,4] = Geospatial.iloc[b,2]

In [126]:
toronto.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


# PART: 3

### Install and import all the required libraries

In [111]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


### Check for the Area with Most Neighbourhoods and select it for further action

In [19]:
toronto.Borough.value_counts()

North York          24
Downtown Toronto    18
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [20]:
northyork = toronto[toronto.Borough == 'North York'].reset_index(drop = True)
northyork.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.8038,-79.3635
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.7785,-79.3466
2,M2K,North York,Bayview Village,43.7869,-79.386
3,M2L,North York,"Silver Hills, York Mills",43.7575,-79.3747
4,M2M,North York,"Newtonbrook, Willowdale",43.7891,-79.4085


### Use Geocode to get the coordinates of North York and Start Exploring it's nearby areas using Foursquare API

In [21]:
address = 'North York, Toronto, Ontario'
locator = Nominatim(user_agent ='toronto-map_data')
location = locator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of ', address,' are: ',latitude, ' and ',longitude)

The coordinates of  North York, Toronto, Ontario  are:  43.7708175  and  -79.4132998


In [24]:
CLIENT_ID = 'PBEJO0ENPM4XXVANUH02NIDL4IJC4PSSH1QFBUFQ0DJRVTVG' # your Foursquare ID
CLIENT_SECRET = 'PKIGPFC4LDXYTZSZZF2CYTOQRSXXT2RUVA23OJMN22N2QGF5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PBEJO0ENPM4XXVANUH02NIDL4IJC4PSSH1QFBUFQ0DJRVTVG
CLIENT_SECRET:PKIGPFC4LDXYTZSZZF2CYTOQRSXXT2RUVA23OJMN22N2QGF5


In [26]:
LIMIT = 50

radius = 500

### Function to get names, categories and coordinates of nearby places in the Neighbourhoods(Postal Codes) of the North York Borough.

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
northyork_venues = getNearbyVenues(names=northyork['Postcode'],
                                   latitudes=northyork['Latitude'],
                                   longitudes=northyork['Longitude']
                                  )


M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M5M
M6A
M6B
M6L
M9L
M9M


In [31]:
northyork_venues.head()

,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2H,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,M2H,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,M2H,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,M2H,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,M2H,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


#### One hot encoding according to each category of nearby venues

In [33]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Postcode'] = northyork_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Postcode,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
northyork_grouped = northyork_onehot.groupby('Postcode').mean().reset_index()

In [35]:
northyork_grouped.head()

,Postcode,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,M2H,0.0,0.0,0.00,0.0,0.00,0.166667,0.00,0.00,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
1,M2J,0.0,0.0,0.02,0.0,0.02,0.000000,0.02,0.02,0.0,...,0.0,0.02,0.0,0.02,0.02,0.02,0.0,0.0,0.02,0.02
2,M2K,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,0.25,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
3,M2L,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00
4,M2M,0.0,0.0,0.00,0.0,0.00,0.000000,0.00,0.00,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00


#### Function to find the most common venues

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = northyork_grouped['Postcode']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,Dog Run,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant
1,M2J,Clothing Store,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Shoe Store
2,M2K,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store
3,M2L,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store
4,M2M,Park,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store


### Perform K-Means Clustering and Visualize our Data

In [41]:
northyork_clustering = northyork_grouped.drop('Postcode', axis = 1)

In [103]:
k_clusters = 3
kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(northyork_clustering)
kmeans.labels_

array([2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0,
       2, 2], dtype=int32)

In [105]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#neighborhoods_venues_sorted.drop(['Cluster Labels'], axis = 1, inplace = True)
northyork_merged = northyork

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

northyork_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M2H,North York,Hillcrest Village,43.8038,-79.3635,2,Dog Run,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.7785,-79.3466,2,Clothing Store,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Shoe Store
2,M2K,North York,Bayview Village,43.7869,-79.386,2,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store
3,M2L,North York,"Silver Hills, York Mills",43.7575,-79.3747,1,Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store
4,M2M,North York,"Newtonbrook, Willowdale",43.7891,-79.4085,0,Park,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store


In [106]:
# create map
northyork_clustermap = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = ['red','blue','green','yellow','black','pink', 'darkgreen','cyan','forestgreen']

# add markers to the map
markers_colors = []
for lat, lon, post, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Postcode'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(post) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(northyork_clustermap)
       
northyork_clustermap

### Observations from the clusters to find something meaningful

#### Here in Cluster = 0, we find that there is lots of park area and Landscaping sights.

In [107]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1,2]+list(range(6, northyork_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,North York,"Newtonbrook, Willowdale",Park,Women's Store,Electronics Store,Construction & Landscaping,Convenience Store
6,North York,York Mills West,Park,Convenience Store,Bank,Women's Store,Electronics Store
8,North York,Parkwoods,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Dog Run
13,North York,"CFB Toronto, Downsview East",Airport,Park,Women's Store,Electronics Store,Construction & Landscaping
14,North York,Downsview West,Grocery Store,Park,Shopping Mall,Bank,Comfort Food Restaurant
21,North York,"Downsview, North Park, Upwood Park",Park,Construction & Landscaping,Bakery,Basketball Court,Empanada Restaurant


#### In Cluster = 1, there is not enough data to come to a conclusion.

In [108]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1,2]+list(range(6, northyork_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,North York,"Silver Hills, York Mills",Cafeteria,Women's Store,Coffee Shop,Construction & Landscaping,Convenience Store


#### In Cluster = 2, we can see that there are lots of food and beverage venues.

In [109]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1,2]+list(range(6, northyork_merged.shape[1]))]]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,Hillcrest Village,Dog Run,Golf Course,Pool,Athletics & Sports,Mediterranean Restaurant
1,North York,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Shoe Store
2,North York,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store
5,North York,Willowdale South,Ramen Restaurant,Coffee Shop,Café,Sandwich Place,Pizza Place
7,North York,Willowdale West,Pharmacy,Discount Store,Pizza Place,Butcher,Coffee Shop
9,North York,Don Mills North,Japanese Restaurant,Caribbean Restaurant,Gym / Fitness Center,Café,Basketball Court
10,North York,"Flemingdon Park, Don Mills South",Gym,Coffee Shop,Beer Store,Discount Store,Japanese Restaurant
11,North York,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Gift Shop,Grocery Store,Middle Eastern Restaurant,Pet Store
12,North York,"Northwood Park, York University",Coffee Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar
15,North York,Downsview Central,Food Truck,Home Service,Baseball Field,Women's Store,Electronics Store
